### Importing libraries

In [1]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader 
from scipy.stats import zscore
from sklearn import metrics
import random
from surprise.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

#accessing the model and memory based techniques
from surprise import SVD
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore 
from surprise import KNNBaseline
from surprise import BaselineOnly
from surprise import CoClustering

#accessing the similarity metrics
from math import sqrt
from surprise import accuracy
from surprise.model_selection.validation import cross_validate

### Accessing the data

In [2]:
#defining the file path 
file_path = './cleanedDatasets/cleaned_user_ratings.csv'

#reading the data using pandas library
user_ratings_df = pd.read_csv(file_path, encoding= 'unicode_escape')
user_ratings_df

,course_id,user_id,rating
0,2001,1001,5
1,2001,1002,3
2,2001,1003,1
3,2001,1004,0
4,2001,1005,2
...,...,...,...
8472,2424,1016,5
8473,2424,1017,4
8474,2424,1018,1
8475,2424,1019,2


### Preparing the data in a suitable format

In [3]:
#creating a reader object
#line format defines the order to match with the file - since reading with pandas not needed
#sep is the comma in this case since it is a csv file - since reading with pandas not needed
#rating_scale it is a tuple defining the lowest and highest possible range, if a number is not included in the tuple it will be ignored. 
reader = Reader(rating_scale=(0, 5))

#importing the data in a fromat surprise to be able to work with it - by loading it from the pandas data frame
#data = Dataset.load_from_df(user_ratings_df[['course_id','user_id','rating']], reader=reader) #------> did not work as the order impacted the functionality  for the KNNBasic user-based 
data = Dataset.load_from_df(user_ratings_df[['user_id','course_id','rating']], reader=reader)

In [4]:
data

##### Splitting the data 

In [5]:
raw_ratings_data = data.raw_ratings #lists the raw ratings in the following order: user_id, course_id, rating, and the timestamp

In [6]:
data.raw_ratings

[(1001, 2001, 5.0, None),
 (1002, 2001, 3.0, None),
 (1003, 2001, 1.0, None),
 (1004, 2001, 0.0, None),
 (1005, 2001, 2.0, None),
 (1006, 2001, 1.0, None),
 (1007, 2001, 0.0, None),
 (1008, 2001, 0.0, None),
 (1009, 2001, 0.0, None),
 (1010, 2001, 0.0, None),
 (1011, 2001, 3.0, None),
 (1012, 2001, 0.0, None),
 (1013, 2001, 0.0, None),
 (1014, 2001, 4.0, None),
 (1015, 2001, 1.0, None),
 (1016, 2001, 0.0, None),
 (1017, 2001, 2.0, None),
 (1018, 2001, 0.0, None),
 (1019, 2001, 1.0, None),
 (1020, 2001, 3.0, None),
 (1001, 2002, 3.0, None),
 (1002, 2002, 5.0, None),
 (1003, 2002, 0.0, None),
 (1004, 2002, 2.0, None),
 (1005, 2002, 1.0, None),
 (1006, 2002, 0.0, None),
 (1007, 2002, 4.0, None),
 (1008, 2002, 5.0, None),
 (1009, 2002, 0.0, None),
 (1010, 2002, 1.0, None),
 (1011, 2002, 0.0, None),
 (1012, 2002, 3.0, None),
 (1013, 2002, 2.0, None),
 (1014, 2002, 0.0, None),
 (1015, 2002, 5.0, None),
 (1016, 2002, 2.0, None),
 (1017, 2002, 0.0, None),
 (1018, 2002, 3.0, None),
 (1019, 2002

In [7]:
#before splitting the data it would be ideal to shuffle 
random.shuffle(raw_ratings_data) 

In [8]:
data.raw_ratings #to confirm that the data is shuffled properly, it was compared to the cleaned user rating csv file - note the order of columns is different of course than the orignal file

[(1016, 2415, 4.0, None),
 (1009, 2191, 0.0, None),
 (1019, 2197, 2.0, None),
 (1020, 2051, 0.0, None),
 (1020, 2363, 2.0, None),
 (1014, 2406, 4.0, None),
 (1009, 2153, 2.0, None),
 (1007, 2215, 5.0, None),
 (1011, 2361, 3.0, None),
 (1011, 2369, 4.0, None),
 (1002, 2294, 5.0, None),
 (1006, 2241, 3.0, None),
 (1004, 2111, 2.0, None),
 (1004, 2048, 4.0, None),
 (1006, 2065, 4.0, None),
 (1003, 2058, 5.0, None),
 (1004, 2016, 2.0, None),
 (1016, 2075, 3.0, None),
 (1011, 2195, 1.0, None),
 (1007, 2034, 5.0, None),
 (1002, 2182, 5.0, None),
 (1016, 2277, 4.0, None),
 (1012, 2414, 0.0, None),
 (1011, 2328, 4.0, None),
 (1010, 2118, 4.0, None),
 (1006, 2165, 0.0, None),
 (1002, 2183, 1.0, None),
 (1005, 2029, 5.0, None),
 (1012, 2017, 2.0, None),
 (1002, 2342, 0.0, None),
 (1011, 2222, 4.0, None),
 (1006, 2038, 1.0, None),
 (1002, 2389, 0.0, None),
 (1004, 2352, 0.0, None),
 (1004, 2390, 0.0, None),
 (1012, 2237, 3.0, None),
 (1007, 2294, 4.0, None),
 (1007, 2198, 4.0, None),
 (1017, 2179

In [9]:
#20% assigned to the testing set 
#train_data_raw, test_data_raw = train_test_split(raw_ratings_data, test_size=0.2) # -------> did not work when trying to use the predict on the test set for the KNN Basic method since there were too many values to unpack
ratio = int(len(raw_ratings_data)*0.8)

#assigning data to the training set
train_data_raw = raw_ratings_data[:ratio] 

#assigning data to the testing set
test_data_raw = raw_ratings_data[ratio:] 

# using the entire data to set the training dataset
data.raw_ratings = train_data_raw       
trainset = data.build_full_trainset() 
testset = data.construct_testset(test_data_raw)

In [10]:
# Calculate the lengths (i.e., the number of data points) of the training and testing sets
train_data_length = len(train_data_raw)
test_data_length = len(test_data_raw)

# Print out the sizes of the training and testing data sets
print(f"Number of data points in training set: {train_data_length}")
print(f"Number of data points in testing set: {test_data_length}")

#showing this as a percentage of the total data
total_data_length = train_data_length + test_data_length
train_data_percentage = (train_data_length / total_data_length) * 100
test_data_percentage = (test_data_length / total_data_length) * 100

print(f"Percentage of data in training set: {train_data_percentage:.2f}%")
print(f"Percentage of data in testing set: {test_data_percentage:.2f}%")

Number of data points in training set: 6781
Number of data points in testing set: 1696
Percentage of data in training set: 79.99%
Percentage of data in testing set: 20.01%


### Defining the different techniques for collaborative filtering to determine which one is best

In [11]:
#create dict for different memory-based  and model based recommendation algorithms
recommendation_algorithms =[KNNBasic(), KNNBaseline(), KNNWithMeans(), KNNWithZScore(), SVD(), BaselineOnly(), CoClustering() ] 
results = {} #to store the scores

### Cross-validation performed on all recommendation algorithms

Hyper parameters of cross validation method:

- cv - defines the type of folding for the model we want to use
- n-jobs - determines the number of folds evaluated in parallel. 

In [12]:
for algorithm in recommendation_algorithms:
    #kfold set to 5
    #using three different metrics for evaluation: Mean Absolute Error ,Mean Squared Error, Root Mean Squared Error
    crossval_scores = cross_validate(algorithm, data, measures=["MAE", "MSE", "RMSE"], cv=5, n_jobs=-1)  
    
    #saving and renaming appropraitely
    result = pd.DataFrame.from_dict(crossval_scores).mean(axis=0).\
             rename({'test_mae':'MAE', 'test_mse': 'MSE', 'test_rmse': 'RMSE', 'fit_time': 'Fit Time', 'test_time': 'Test Time'})
    results[str(algorithm).split("algorithms.")[1].split("object ")[0]] = result
    
#printing all models results
all_models = pd.DataFrame.from_dict(results)
all_models.T.sort_values(by='RMSE') #models sorted by RMSE


,MAE,MSE,RMSE,Fit Time,Test Time
matrix_factorization.SVD,1.097270,1.833105,1.353607,0.085245,0.011967
knns.KNNWithZScore,1.279754,2.414580,1.553755,0.011378,0.047979
knns.KNNBaseline,1.283397,2.419144,1.555026,0.007007,0.041491
knns.KNNWithMeans,1.285100,2.430043,1.558648,0.006887,0.032899
co_clustering.CoClustering,1.268170,2.457313,1.567068,0.125411,0.008964
knns.KNNBasic,1.323785,2.540636,1.593584,0.005365,0.025433
baseline_only.BaselineOnly,1.366575,2.633734,1.622723,0.004833,0.004454


### Testing with different parameters

 Preparing the dataset to be used in a way that cross-validation can be applied directly on the entire dataset without the need for a separate test set initially defined 

In [13]:
#using cross-validation to help us test how well the model works accross different subsets of the data
trainset = data.build_full_trainset() #using all the ratings

#finding out how many users and courses we have in the dataset
print('Number of users: ', trainset.n_users, '\n') #20 users that rated
print('Number of items: ', trainset.n_items, '\n') #424 number of courses

Number of users:  20 

Number of items:  424 



#### Hyperparameters for the KNN models

- K parameter - indicates the the upper limit of similar items we want the algorithm to consider. For example in this case if the user rated 21 courses, but k is set to 5, when estimating a rating of a new course which has not yet been rated, only 5 out of the 21 courses that are the closest to the new course will be considered. 

- Similarity option, that defines the way to calculate it. All the similarity functions will return a number between 0 and 1 to a specific (i, j) item pair. 1 means the ratings are perfectly aligned, 0 means there is no connection between the two items. 

Three similarity metrics in the surprise similarity module: cosine, msd,pearson.

Both User-Based an Item-Based are implemented.

User-based collaborative filtering: a technique that predicts what the user might like based on the basis of ratings provided to the items by other users who have similar tastes with the target user. 

Item-based collaborative filtering: is a way to suggest things like products by looking at how similar two items are. Instead of comparing what the items are like (their features or properties), it looks at how users feel about the items—meaning, if people who like one item also tend to like another, those items are considered similar



#### KNNBasic

##### KNNBasic: User-Based Cosine

In [14]:
#fitting
#using user-based cosine similarity
params = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between users
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the cosine similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.6154
MSE: 2.6096
MAE:  1.3451

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.08   {'actual_k': 14, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.0777276242167568, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.1582778919611574, details={'actual_k': 14, 'was_impossible': False})]

##### KNNBasic: Item-Based Cosine

In [15]:
# #build train and train train set
# trainset = data.build_full_trainset()

# #fitting

# params = {
#     "name": "cosine",
#     "user_based": False,  # Compute  similarities between users - False since we want to compare items
# }
# algo = KNNBasic(sim_options = params)
# algo.fit(trainset) #--------------> had to comment this entire section as ZeroDivisionError was displayed

# #test the test set using .test() 
# print('\nAccuracy on the testset:')
# accuracy.rmse(algo.test(testset))
# accuracy.mse(algo.test(testset))
# accuracy.mae(algo.test(testset))

# print('\nPredict Tests: ')
# print(algo.predict(1001, 2001))
# print(algo.predict(2001, 1001))
# print(algo.predict(1001, 88))
# print(algo.predict(1001, 5))

# print('\nPredict Using TestSet list: ')
# testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
# algo.test(testset)[:2]

##### KNNBasic: User-Based MSD 

In [16]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based MSD similarity
params = {
    "name": "msd",
    "user_based": True,  # Compute  similarities between users
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.2239
MSE: 1.4978
MAE:  1.0094

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.08   {'actual_k': 14, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.0802793900842056, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.6444873826783317, details={'actual_k': 14, 'was_impossible': False})]

##### KNNBasic: Item-Based MSD

In [17]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#not using user-based MSD similarity
params = {
    "name": "msd",
    "user_based": False,  # Compute  similarities between users
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.7194
MSE: 0.5176
MAE:  0.4545

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 2.72   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=2.7214942708842385, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=2.6809344387243743, details={'actual_k': 40, 'was_impossible': False})]

##### KNNBasic: User-Based Pearson

In [18]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based pearson similarity
params = {
    "name": "pearson",
    "user_based": True,  # Compute  similarities between users
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the pearson similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.0649
MSE: 1.1339
MAE:  0.8483

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.12   {'actual_k': 13, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.124571586801598, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.7039653827951435, details={'actual_k': 12, 'was_impossible': False})]

##### KNNBasic: Item-Based Pearson

In [19]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#not using user-based pearson similarity
params = {
    "name": "pearson",
    "user_based": False,  # Compute  similarities between users
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the pearson similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.9132
MSE: 0.8340
MAE:  0.6041

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 2.47   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=2.47473060832653, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=4.293223811261339, details={'actual_k': 40, 'was_impossible': False})]

##### KNNBasic: Cosine, User-Based, Shrinkage

In [20]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "cosine",
    "user_based": True,
    "shrinkage": 0 #no shrinkage - might not effect since using consine and only impacts pearson
}
algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the cosine similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.5150
MSE: 2.2951
MAE:  1.2620

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.08   {'actual_k': 14, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.0777276242167568, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.1582778919611574, details={'actual_k': 14, 'was_impossible': False})]

In [21]:
#-------> Commented due to the zero division error

# #build train and train train set
# trainset = data.build_full_trainset()

# #fitting
# #using user-based cosine similarity
# params = {
#     "name": "cosine",
#     "user_based": False,
#     "shrinkage": 0 #no shrinkage - might not effect since using consine and only impacts pearson
# }
# algo = KNNBasic(sim_options = params)
# algo.fit(trainset)

# #test the test set using .test() 
# print('\nAccuracy on the testset:')
# accuracy.rmse(algo.test(testset))
# accuracy.mse(algo.test(testset))
# accuracy.mae(algo.test(testset))

# print('\nPredict Tests: ')
# print(algo.predict(1001, 2001))
# print(algo.predict(2001, 1001))
# print(algo.predict(1001, 88))
# print(algo.predict(1001, 5))

# print('\nPredict Using TestSet list: ')
# testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
# algo.test(testset)[:2]

##### KNNBasic: MSD, User-Based, Shrinkage

In [22]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "msd",
    "user_based": True,
    "shrinkage": 0 #shrinkage is 0 -indicates that no shrinkage adjustment is being applied in this configuration.
}

algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.2239
MSE: 1.4978
MAE:  1.0094

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.08   {'actual_k': 14, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.0802793900842056, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.6444873826783317, details={'actual_k': 14, 'was_impossible': False})]

In [23]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "msd",
    "user_based": False,
    "shrinkage": 0 #shrinkage is 0 -indicates that no shrinkage adjustment is being applied in this configuration.
     # However, it's important to note that shrinkage is specifically mentioned as relevant for "pearson_baseline" similarity. 
     # the "shrinkage" setting here might not have any effect because it's not applicable to the "msd" similarity measure. 
}

algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.7194
MSE: 0.5176
MAE:  0.4545

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 2.72   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=2.7214942708842385, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=2.6809344387243743, details={'actual_k': 40, 'was_impossible': False})]

##### KNNBasic: Pearson, User-Based, Shrinkage

In [24]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "pearson",
    "user_based": True,
    "shrinkage": 0
}

algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the pearson similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.0649
MSE: 1.1339
MAE:  0.8483

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.12   {'actual_k': 13, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.124571586801598, details={'actual_k': 13, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.7039653827951435, details={'actual_k': 12, 'was_impossible': False})]

In [25]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
params = {
    "name": "pearson",
    "user_based": False,
    "shrinkage": 0
}

algo = KNNBasic(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the pearson similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.9132
MSE: 0.8340
MAE:  0.6041

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 2.47   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=2.47473060832653, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=4.293223811261339, details={'actual_k': 40, 'was_impossible': False})]

##### Applying GridSearch

In [26]:
params_grid = {'sim_options' : {'method': ['als','sgd'],
                            "n_epochs" : [5, 10, 20],
                            "user_based": [True, False],
                            "min_support": [3, 5, 8],
                          "user_based": [True, False]}}

grid_search_KNNBasic = GridSearchCV(KNNBasic, params_grid, measures=['RMSE','MAE'], cv=5, n_jobs=-1)                               
grid_search_KNNBasic.fit(data)

print(f'\nRMSE Best Parameters: {grid_search_KNNBasic.best_params["rmse"]}')
print(f'RMSE Best Score: {grid_search_KNNBasic.best_score["rmse"]}')
print(f'MAE Best Parameters: {grid_search_KNNBasic.best_params["mae"]}')
print(f'MAE Best Score: {grid_search_KNNBasic.best_score["mae"]}')


RMSE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'min_support': 3}}
RMSE Best Score: 1.1001987333202565
MAE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'min_support': 3}}
MAE Best Score: 0.7910198204125138


Applying the hyperparameters found in grid_search_KNNBasic 

In [27]:
finalKBL = KNNBaseline(method = "als", n_epochs = 5, user_based = False, reg_u = 10, reg_i = 8)
predictions = finalKBL.fit(trainset).test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.

Updated Accuracy: 
RMSE: 1.1970
MSE: 1.4327
MAE:  0.9806


0.9806180960940517

#### KNN With K-Means

In [28]:
algo = KNNWithMeans()

#applying cross validation 
#kfold set to 5
cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

#build train and train train set
trainset = data.build_full_trainset()
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Evaluating RMSE, MAE of algorithm KNNWithMeans on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.5598  1.5756  1.5632  1.5538  1.5479  1.5601  0.0094  
MAE (testset)     1.2787  1.3033  1.2864  1.2825  1.2864  1.2875  0.0084  
Fit time          0.01    0.00    0.01    0.00    0.01    0.01    0.00    
Test time         0.04    0.03    0.03    0.02    0.02    0.03    0.01    
Computing the msd similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 1.1971
MSE: 1.4330
MAE:  0.9800

Predict Tests: 
user: 1001       item: 2001       r_ui = No

[Prediction(uid=1001, iid=2001, r_ui=5, est=1.018160353955444, details={'actual_k': 14, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.9001831677217638, details={'actual_k': 14, 'was_impossible': False})]

##### KNNWithMeans: User-based, Cosine 

In [29]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
parans = {
    "name": "cosine",
    "user_based": True,  # Compute  similarities between users
}
algo = KNNWithMeans(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the pearson similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.7655
MSE: 0.5859
MAE:  0.5144

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.34   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.33717557328254, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=3.1536618392391267, details={'actual_k': 40, 'was_impossible': False})]

##### KNNWithMeans: Item-based Cosine 

In [30]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based cosine similarity
parans = {
    "name": "cosine",
    "user_based": False,  # Compute  similarities between items
}
algo = KNNWithMeans(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the pearson similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.7655
MSE: 0.5859
MAE:  0.5144

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.34   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.33717557328254, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=3.1536618392391267, details={'actual_k': 40, 'was_impossible': False})]

##### KNNWithMeans: User-based MSD 

In [31]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using user-based msd similarity
parans = {
    "name": "msd",
    "user_based": True,  # Compute similarities between users
}
algo = KNNWithMeans(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the pearson similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.7655
MSE: 0.5859
MAE:  0.5144

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.34   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.33717557328254, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=3.1536618392391267, details={'actual_k': 40, 'was_impossible': False})]

##### KNNWithMeans: Item-based MSD 

In [32]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using item-based msd similarity
parans = {
    "name": "msd",
    "user_based": False,  # Compute similarities between items
}
algo = KNNWithMeans(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the pearson similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.7655
MSE: 0.5859
MAE:  0.5144

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.34   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.33717557328254, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=3.1536618392391267, details={'actual_k': 40, 'was_impossible': False})]

##### KNNWithMeans: User-based Pearson 

In [33]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using item-based msd similarity
parans = {
    "name": "pearson",
    "user_based": True,  # Compute similarities between items
}
algo = KNNWithMeans(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the pearson similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.7655
MSE: 0.5859
MAE:  0.5144

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.34   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.33717557328254, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=3.1536618392391267, details={'actual_k': 40, 'was_impossible': False})]

##### KNNWithMeans: Item-based Pearson 

In [34]:
#build train and train train set
trainset = data.build_full_trainset()

#fitting
#using item-based msd similarity
parans = {
    "name": "pearson",
    "user_based": False,  # Compute similarities between items
}
algo = KNNWithMeans(sim_options = params)
algo.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(algo.test(testset))
accuracy.mse(algo.test(testset))
accuracy.mae(algo.test(testset))

print('\nPredict Tests: ')
print(algo.predict(1001, 2001))
print(algo.predict(2001, 1001))
print(algo.predict(1001, 88))
print(algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
algo.test(testset)[:2]

Computing the pearson similarity matrix...
Done computing similarity matrix.

Accuracy on the testset:
RMSE: 0.7655
MSE: 0.5859
MAE:  0.5144

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.34   {'actual_k': 40, 'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': True, 'reason': 'User and/or item is unknown.'}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.33717557328254, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=3.1536618392391267, details={'actual_k': 40, 'was_impossible': False})]

##### KNNWithMeans - Applying the Grid Search

In [35]:
#doing this with name is msd or cosine causes float or zero division error
params = {'sim_options' : {'method': ['als','sgd'],
                            "n_epochs" : [5, 10, 20],
                            "user_based": [True, False],
                            "min_support": [3, 5, 8],
                          "user_based": [True, False]}}

gsKWM = GridSearchCV(KNNWithMeans, params, measures=['mae', 'rmse'], cv=5, n_jobs=-1)                               
gsKWM.fit(data)

print(f'\nRMSE Best Parameters: {gsKWM.best_params["rmse"]}')
print(f'RMSE Best Score: {gsKWM.best_score["rmse"]}')
print(f'MAE Best Parameters: {gsKWM.best_params["mae"]}')
print(f'MAE Best Score: {gsKWM.best_score["mae"]}')


RMSE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'min_support': 3}}
RMSE Best Score: 1.1280282858133623
MAE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'min_support': 3}}
MAE Best Score: 0.845871717264185


##### Applying the best hyperparameters for it 

In [36]:
finalKWM = KNNWithMeans(method = "als", n_epochs = 5, user_based = False, min_support = 3)
predictions = finalKWM.fit(trainset).test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.

Updated Accuracy: 
RMSE: 1.1971
MSE: 1.4330
MAE:  0.9800


0.9799949858706068

#### KNN With Z Score

##### Applying the Grid Search

In [37]:
params = {'sim_options' : {'method': ['als','sgd'],
                            "n_epochs" : [5, 10, 20],
                            "user_based": [True, False],
                            "reg_u": [10, 12, 15],
                            "reg_i": [8, 10, 12],
                            "learning_rate": [0.0005, 0.05]}}

gsKZS = GridSearchCV(KNNWithZScore, params, measures=['mae', 'rmse'], cv=5, n_jobs=-1)                               
gsKZS.fit(data)

print(f'\nRMSE Best Parameters: {gsKZS.best_params["rmse"]}')
print(f'RMSE Best Score: {gsKZS.best_score["rmse"]}')
print(f'MAE Best Parameters: {gsKZS.best_params["mae"]}')
print(f'MAE Best Score: {gsKZS.best_score["mae"]}')


RMSE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'reg_u': 10, 'reg_i': 8, 'learning_rate': 0.0005}}
RMSE Best Score: 1.1329436940531692
MAE Best Parameters: {'sim_options': {'method': 'als', 'n_epochs': 5, 'user_based': False, 'reg_u': 10, 'reg_i': 8, 'learning_rate': 0.0005}}
MAE Best Score: 0.8461152856588928


##### Applying the best parameters found

In [38]:
finalKZS = KNNWithZScore(method = "als", n_epochs = 5, user_based = False, reg_u = 10, reg_i = 8, learning_rate = 0.0005)
predictions = finalKZS.fit(trainset).test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)

Computing the msd similarity matrix...
Done computing similarity matrix.

Updated Accuracy: 
RMSE: 1.1979
MSE: 1.4350
MAE:  0.9793


0.9793229112434899

#### SVD 

In [39]:
#algo
svd = SVD()

#cross validate with kfold set to 5
cross_validate(svd, data, measures = ['RMSE', 'MAE'], cv = 5, verbose = True)

#build train and train train set
trainset = data.build_full_trainset()
svd.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(svd.test(testset))
accuracy.mse(svd.test(testset))
accuracy.mae(svd.test(testset))

print('\nPredict Tests: ')
print(svd.predict(1001, 2001))
print(svd.predict(2001, 1001))
print(svd.predict(1001, 88))
print(svd.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
svd.test(testset)[:2]

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.3738  1.3538  1.3287  1.3356  1.3674  1.3519  0.0175  
MAE (testset)     1.1051  1.0894  1.0796  1.0959  1.1044  1.0949  0.0096  
Fit time          0.08    0.05    0.05    0.05    0.05    0.05    0.01    
Test time         0.01    0.00    0.00    0.01    0.00    0.00    0.00    

Accuracy on the testset:
RMSE: 0.7750
MSE: 0.6007
MAE:  0.5534

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.02   {'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': False}
user: 1001       item: 88         r_ui = None   est = 2.42   {'was_impossible': False}
user: 1001       item: 5          r_ui = None   est = 2.42   {'was_impossible': False}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.023094868725199, details={'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=2.4594113858497892, details={'was_impossible': False})]

##### Applying Grid Search

In [40]:
params = {"n_factors": range(10, 100, 20),
         "n_epochs": [5, 10, 20],
         "lr_all": [0.002, 0.005],
         "reg_all": [0.2, 0.5]}

gsSVD = GridSearchCV(SVD, params, measures = ["RMSE", "MAE"], cv = 5, n_jobs = -1)
gsSVD.fit(data)

print(f'\nRMSE Best Parameters: {gsSVD.best_params["rmse"]}')
print(f'RMSE Best Score: {gsSVD.best_score["rmse"]}')
print(f'MAE Best Parameters: {gsSVD.best_params["mae"]}')
print(f'MAE Best Score: {gsSVD.best_score["mae"]}')


RMSE Best Parameters: {'n_factors': 90, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}
RMSE Best Score: 1.4845021068177737
MAE Best Parameters: {'n_factors': 90, 'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.2}
MAE Best Score: 1.2510150099922321


##### Applying the best parameters

In [41]:
finalSVD = SVD(n_factors = 90, n_epochs = 20, lr_all = 0.005, reg_all = 0.2)
predictions = finalSVD.fit(trainset).test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions)
accuracy.mae(predictions)


Updated Accuracy: 
RMSE: 1.2910
MSE: 1.6666
MAE:  1.0783


1.078297429320599

#### BaselineOnly

In [42]:
# Initialize the BaselineOnly algorithm
baseline_algo = BaselineOnly()

# Cross validate with k-fold set to 5
cross_validate(baseline_algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


trainset = data.build_full_trainset()
baseline_algo.fit(trainset)


#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(baseline_algo.test(testset))
accuracy.mse(baseline_algo.test(testset))
accuracy.mae(baseline_algo.test(testset))

print('\nPredict Tests: ')
print(baseline_algo.predict(1001, 2001))
print(baseline_algo.predict(2001, 1001))
print(baseline_algo.predict(1001, 88))
print(baseline_algo.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
baseline_algo.test(testset)[:2]


Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Evaluating RMSE, MAE of algorithm BaselineOnly on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6358  1.6208  1.5851  1.6484  1.6171  1.6214  0.0213  
MAE (testset)     1.3717  1.3614  1.3316  1.3922  1.3619  1.3637  0.0196  
Fit time          0.01    0.01    0.00    0.01    0.00    0.01    0.00    
Test time         0.00    0.00    0.01    0.00    0.01    0.00    0.00    
Estimating biases using als...

Accuracy on the testset:
RMSE: 1.5624
MSE: 2.4410
MAE:  1.3137

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.58   {'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': False}
user: 1001       item: 88         r_ui = None   est = 2.43   {'was_impossible': False}
user: 1001       item: 5          

[Prediction(uid=1001, iid=2001, r_ui=5, est=1.582962128031363, details={'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=1.9633962293293656, details={'was_impossible': False})]

##### Applying the Grid Search

In [43]:
# Define the parameter grid
params = {'bsl_options': {
                    'method': ['als', 'sgd'],  # Alternating Least Squares (ALS) and Stochastic Gradient Descent (SGD)
                    'n_epochs': [5, 10, 20],   # Only used for SGD
                    'reg_u': [12, 15, 17],     # Regularization parameter for users, only for ALS
                    'reg_i': [5, 10, 15]       # Regularization parameter for items, only for ALS
                }}

gs_baseline = GridSearchCV(BaselineOnly, params, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1)
gs_baseline.fit(data)

# Print best scores
print(f'\nRMSE Best Parameters: {gs_baseline.best_params["rmse"]}')
print(f'RMSE Best Score: {gs_baseline.best_score["rmse"]}')
print(f'MAE Best Parameters: {gs_baseline.best_params["mae"]}')
print(f'MAE Best Score: {gs_baseline.best_score["mae"]}')



RMSE Best Parameters: {'bsl_options': {'method': 'als', 'n_epochs': 20, 'reg_u': 17, 'reg_i': 10}}
RMSE Best Score: 1.619736022927086
MAE Best Parameters: {'bsl_options': {'method': 'als', 'n_epochs': 5, 'reg_u': 12, 'reg_i': 5}}
MAE Best Score: 1.3547772296747858


##### Applying the best parameters

In [44]:
# Train and test with the best found parameters
final_baseline = BaselineOnly(bsl_options=gs_baseline.best_params['rmse']['bsl_options'])
final_baseline.fit(trainset)
predictions = final_baseline.test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions, verbose=True)
accuracy.mae(predictions, verbose=True)

Estimating biases using als...

Updated Accuracy: 
RMSE: 1.5624
MSE: 2.4410
MAE:  1.3139


1.3138841373524353

#### CoClustering

In [45]:
# Initialize the CoClustering algorithm
co_clustering = CoClustering()

# Cross validate with k-fold set to 5
cross_validate(co_clustering, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

trainset = data.build_full_trainset()
co_clustering.fit(trainset)

#test the test set using .test() 
print('\nAccuracy on the testset:')
accuracy.rmse(co_clustering.test(testset))
accuracy.mse(co_clustering.test(testset))
accuracy.mae(co_clustering.test(testset))

print('\nPredict Tests: ')
print(co_clustering.predict(1001, 2001))
print(co_clustering.predict(2001, 1001))
print(co_clustering.predict(1001, 88))
print(co_clustering.predict(1001, 5))

print('\nPredict Using TestSet list: ')
testset = [data.df.loc[i].to_list() for i in range(len(data.df))]
co_clustering.test(testset)[:2]

Evaluating RMSE, MAE of algorithm CoClustering on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.6953  1.5559  1.5333  1.5149  1.5659  1.5731  0.0636  
MAE (testset)     1.3778  1.2431  1.2434  1.2409  1.2692  1.2749  0.0525  
Fit time          0.11    0.10    0.10    0.08    0.09    0.10    0.01    
Test time         0.01    0.00    0.01    0.01    0.01    0.01    0.00    

Accuracy on the testset:
RMSE: 1.3737
MSE: 1.8871
MAE:  1.1043

Predict Tests: 
user: 1001       item: 2001       r_ui = None   est = 1.36   {'was_impossible': False}
user: 2001       item: 1001       r_ui = None   est = 2.59   {'was_impossible': False}
user: 1001       item: 88         r_ui = None   est = 2.59   {'was_impossible': False}
user: 1001       item: 5          r_ui = None   est = 2.59   {'was_impossible': False}

Predict Using TestSet list: 


[Prediction(uid=1001, iid=2001, r_ui=5, est=1.3615862406643697, details={'was_impossible': False}),
 Prediction(uid=1002, iid=2001, r_ui=3, est=2.527029249353864, details={'was_impossible': False})]

##### Applying Grid Search

In [46]:
# Define the parameter grid
params = {
    'n_cltr_u': [3, 5, 7],  # Number of user clusters
    'n_cltr_i': [3, 5, 7],  # Number of item clusters
    'n_epochs': [20, 30, 40]  # Number of epochs
}

# Perform grid search
gs_co_clustering = GridSearchCV(CoClustering, params, measures=['RMSE', 'MAE'], cv=5, n_jobs=-1)
gs_co_clustering.fit(data)

# Print best scores
print(f'\nRMSE Best Parameters: {gs_co_clustering.best_params["rmse"]}')
print(f'RMSE Best Score: {gs_co_clustering.best_score["rmse"]}')
print(f'MAE Best Parameters: {gs_co_clustering.best_params["mae"]}')
print(f'MAE Best Score: {gs_co_clustering.best_score["mae"]}')


RMSE Best Parameters: {'n_cltr_u': 7, 'n_cltr_i': 5, 'n_epochs': 40}
RMSE Best Score: 1.4449439269152546
MAE Best Parameters: {'n_cltr_u': 7, 'n_cltr_i': 5, 'n_epochs': 40}
MAE Best Score: 1.119033065418896


##### Applying the best parameters

In [47]:
# Train and test with the best found parameters
final_co_clustering = CoClustering(n_cltr_u=gs_co_clustering.best_params['rmse']['n_cltr_u'],
                                   n_cltr_i=gs_co_clustering.best_params['rmse']['n_cltr_i'],
                                   n_epochs=gs_co_clustering.best_params['rmse']['n_epochs'])
final_co_clustering.fit(trainset)
predictions = final_co_clustering.test(testset)

print('\nUpdated Accuracy: ')
accuracy.rmse(predictions)
accuracy.mse(predictions, verbose=True)
accuracy.mae(predictions, verbose=True)


Updated Accuracy: 
RMSE: 1.2721
MSE: 1.6181
MAE:  0.9962


0.9962012952617085